In [25]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

In [9]:
dataset = pd.read_csv('dataset.csv')
dataset.head()

,X,Y,Z,A0,A1,A2,A5,A6,A7
0,0.000000,-0.759297,0.419074,3.31,2.01,0.0,1.94,1.58,1.65
1,0.058824,-0.759205,0.418863,3.39,2.23,0.0,2.09,1.73,1.80
2,0.117647,-0.759242,0.419042,3.46,2.16,0.0,2.09,1.80,1.87
3,0.176471,-0.759302,0.419248,3.67,2.30,0.0,2.23,1.87,1.94
4,0.235294,-0.759177,0.418970,3.82,2.30,0.0,2.16,1.87,1.94


In [10]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10708 entries, 0 to 10707
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X       10708 non-null  float64
 1   Y       10708 non-null  float64
 2   Z       10708 non-null  float64
 3   A0      10708 non-null  float64
 4   A1      10708 non-null  float64
 5   A2      10708 non-null  float64
 6   A5      10708 non-null  float64
 7   A6      10708 non-null  float64
 8   A7      10708 non-null  float64
dtypes: float64(9)
memory usage: 753.0 KB


In [18]:
X, y = dataset[["X", "Y", "Z"]], dataset[['A0', 'A1', 'A2', 'A5', 'A6', 'A7']]

x_mean = X.iloc[:, 0].mean()

x_std = X.iloc[:, 0].std()
y_mean = X.iloc[:, 1].mean()
y_std = X.iloc[:, 1].std()
z_mean = X.iloc[:, 2].mean()
z_std = X.iloc[:, 2].std()

# Normalize each column
X_norm = np.copy(X)
X_norm[:, 0] = (X.iloc[:, 0] - x_mean) / x_std
X_norm[:, 1] = (X.iloc[:, 1] - y_mean) / y_std
X_norm[:, 2] = (X.iloc[:, 2] - z_mean) / z_std

y_norm = y / 73.8

In [21]:
X_norm = torch.tensor(pd.DataFrame(X_norm).values, dtype=torch.float32)
y_norm = torch.tensor(pd.DataFrame(y_norm).values, dtype=torch.float32)

X_train, X_temp, y_train, y_temp = train_test_split(X_norm, y_norm, test_size=0.2, shuffle=True, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [22]:
X_train

tensor([[-0.9425,  0.6090,  1.6928],
        [-0.7765,  1.6047,  0.8203],
        [ 1.5648, -0.5537,  0.8535],
        ...,
        [-0.1568, -0.2147, -1.0535],
        [-1.2321, -1.6867,  0.6647],
        [ 0.6153,  0.3961,  0.7643]])

In [26]:
# Create datasets
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

# Create loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [27]:
class LinearlyActuatedStruts(nn.Module):
    def __init__(self):
        super(LinearlyActuatedStruts, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(3, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 6),
            nn.Sigmoid()  # Constrain to [0, 1]
        )
        
    def forward(self, x):
        return self.model(x)

In [28]:
model = LinearlyActuatedStruts()  # assuming you defined the class before
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
    
    train_loss /= len(train_loader.dataset)
    
    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

Epoch 1/100 | Train Loss: 0.0700 | Val Loss: 0.0278
Epoch 2/100 | Train Loss: 0.0189 | Val Loss: 0.0122
Epoch 3/100 | Train Loss: 0.0102 | Val Loss: 0.0079
Epoch 4/100 | Train Loss: 0.0069 | Val Loss: 0.0056
Epoch 5/100 | Train Loss: 0.0035 | Val Loss: 0.0032
Epoch 6/100 | Train Loss: 0.0025 | Val Loss: 0.0022
Epoch 7/100 | Train Loss: 0.0021 | Val Loss: 0.0020
Epoch 8/100 | Train Loss: 0.0018 | Val Loss: 0.0016
Epoch 9/100 | Train Loss: 0.0016 | Val Loss: 0.0015
Epoch 10/100 | Train Loss: 0.0015 | Val Loss: 0.0015
Epoch 11/100 | Train Loss: 0.0014 | Val Loss: 0.0013
Epoch 12/100 | Train Loss: 0.0012 | Val Loss: 0.0012
Epoch 13/100 | Train Loss: 0.0013 | Val Loss: 0.0011
Epoch 14/100 | Train Loss: 0.0010 | Val Loss: 0.0010
Epoch 15/100 | Train Loss: 0.0010 | Val Loss: 0.0009
Epoch 16/100 | Train Loss: 0.0009 | Val Loss: 0.0009
Epoch 17/100 | Train Loss: 0.0009 | Val Loss: 0.0008
Epoch 18/100 | Train Loss: 0.0009 | Val Loss: 0.0012
Epoch 19/100 | Train Loss: 0.0009 | Val Loss: 0.0009
Ep

In [29]:
model.eval()
with torch.no_grad():
    predictions = model(X_test)

# Rescale predictions back to [0, 73.8]
pred_real = predictions * 73.8

In [30]:
y_test_scaled = y_test * 73.8

print(pred_real)

tensor([[7.3800e+01, 7.3797e+01, 7.3800e+01, 9.6888e-01, 3.3849e-02, 6.7670e+01],
        [4.3669e-04, 4.8995e-01, 5.9585e+01, 7.2945e-02, 4.8643e+01, 1.6026e-01],
        [5.2107e+01, 3.0099e+01, 4.4288e-01, 1.5960e+01, 2.3254e+01, 2.2930e+01],
        ...,
        [7.3798e+01, 7.3641e+01, 7.3756e+01, 5.3089e+01, 1.0811e-01, 3.0853e-02],
        [7.3783e+01, 7.3541e+01, 7.3793e+01, 2.5760e-01, 7.3597e+01, 5.3416e+01],
        [7.3702e+01, 6.9238e+01, 8.5314e-03, 4.1192e+01, 6.5381e+01, 5.3757e+01]])


In [31]:
print(y_test_scaled)

tensor([[73.4300, 73.3600, 73.2900,  0.0000,  0.0000, 69.4700],
        [ 0.0000,  0.0000, 57.5600,  0.0000, 48.9800,  0.0000],
        [49.2700, 28.4200,  0.0000, 16.8800, 24.0200, 23.3700],
        ...,
        [73.8000, 73.7200, 73.8000, 54.3900,  0.0000,  0.0000],
        [73.8000, 73.8000, 73.8000,  0.0000, 73.8000, 54.6100],
        [73.5800, 71.1300,  0.0000, 44.6500, 67.7400, 56.7000]])
